<a href="https://colab.research.google.com/github/tomonari-masada/course2024-nlp/blob/main/07_PyTorch_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch入門 (3)

## テキスト分類
* https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html

* モデルを定義するところ以外は、大幅に変えている。
  * トークナイザの訓練から自前でおこなうことにした。
  * データセットはHugging Faceのdatasetsライブラリから使うようにした。


* ランタイムのタイプをGPUに設定しておこう。

## 準備

### インストール

* Hugging Faceのdatasetsライブラリとtokenizersライブラリをインストール

In [ ]:
!pip install datasets tokenizers

## 再現性の確保


* 再現性
  * https://github.com/huggingface/transformers/blob/main/src/transformers/trainer_utils.py

In [ ]:
import os
import random
import torch
import numpy as np

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)

set_seed(0)

## デバイスの設定

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

## データセット

### AG Newsデータセット
* 今回はAG_NEWSというテキスト分類用のデータセットを使う。
  * 4クラス分類問題を解く。

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ag_news")

In [ ]:
dataset

* ラベルの意味は、以下の通り。（ https://huggingface.co/datasets/ag_news を参照。）

In [ ]:
ag_news_label = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tec"}

* 中身を少し見てみる。

In [ ]:
dataset["train"][0]

In [ ]:
dataset["train"]["text"][0]

In [ ]:
dataset["train"]["label"][0]

In [ ]:
print(f"number of different labels: {len(set(dataset['train']['label']))}")

### データセットの分割

In [ ]:
train_valid = dataset["train"].train_test_split(test_size=0.05)
train_valid

In [ ]:
from datasets import DatasetDict

dataset = DatasetDict({
    "train": train_valid["train"],
    "valid": train_valid["test"],
    "test": dataset["test"],
})

In [ ]:
dataset

## トークナイザ
* 以下の説明は、ほぼ次のHugging Faceのdocumentationそのまま。
  * https://huggingface.co/docs/tokenizers/pipeline

### トークン化アルゴリズム
* 今回はWordPieceアルゴリズムを使う。
  * https://huggingface.co/learn/nlp-course/chapter6/6?fw=pt
  * https://huggingface.co/docs/tokenizers/api/models#tokenizers.models.BPE
* 見たことがない文字列は、unknownトークンとして検出する。
  * unknownトークンを避けるには、byteレベルでトークン化すれば良い。
  * だが、今回は、このような高度なトークン化は行わない。
  * byteレベルのトークン化については、
  [ここ](https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt)の緑色のコメント部分を参照。

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece

tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

In [ ]:
tokenizer.model.unk_token

### テキストの正規化
* NFDについては、例えば、下の記事を参照。
  * https://qiita.com/fury00812/items/b98a7f9428d1395fc230
* Lowercase()は小文字化、StripAccents()はアクセント記号の除去。

In [ ]:
from tokenizers import normalizers
from tokenizers.normalizers import NFD, Lowercase, StripAccents

tokenizer.normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])

* このnormalizerがどんな正規化をするか、見てみる。

In [ ]:
tokenizer.normalizer.normalize_str("Héllò hôw are ü?")

### プレトークナイザ
* トークナイザを訓練させるとき、最初に無条件に実行するトークン化を設定する。
* 例えば、英語の場合、まずは無条件に空白文字でトークン化するのが普通。
  * https://huggingface.co/docs/tokenizers/api/pre-tokenizers#tokenizers.pre_tokenizers.Whitespace

In [ ]:
from tokenizers.pre_tokenizers import Whitespace

tokenizer.pre_tokenizer = Whitespace()

### トークナイザのTrainer
* 今回は、特殊トークンの設定を除いて、デフォルトの設定を使う。
  * 特殊トークンは、今回は実際には`[UNK]`しか使わない。
  * このように書けば良いという例として、他の特殊トークンも示しておく。

In [ ]:
from tokenizers.trainers import WordPieceTrainer

trainer = WordPieceTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [ ]:
trainer.vocab_size

In [ ]:
trainer.special_tokens

### トークナイザの訓練
* 語彙集合を決めるときは、訓練データ部分だけを使う。
* trainerを与えるのを忘れないように。
  * trainerを与えるのを忘れると、デフォルトの設定で訓練されてしまう。


In [ ]:
tokenizer.train_from_iterator(dataset["train"]["text"], trainer)

* 訓練したトークナイザは、JSON形式で保存もできる。

In [ ]:
import os

save_dir = "/content/drive/MyDrive/2024courses/nlp"

tokenizer.save(os.path.join(save_dir, "my-tokenizer.json"))

* 語彙サイズ

In [ ]:
tokenizer.get_vocab_size()

* 語彙の取得

In [ ]:
vocab = tokenizer.get_vocab()
print(vocab)

* `[UNK]`トークンが語彙に入っているか確認する。

In [ ]:
tokenizer.model.unk_token in vocab

* 単語トークンの列が整数の列に変換されることを確認する。

In [ ]:
output = tokenizer.encode(dataset["train"]["text"][0])

In [ ]:
print(dataset["train"]["text"][0])

In [ ]:
print(output.ids)

In [ ]:
print(output.tokens)

* offsetsは各トークンが何文字目から何文字目までかを表す。

In [ ]:
print(output.offsets)

* 次に、わざと、トークナイザが見たことなさそうなトークンを含むテキストをトークン化させてみる。

In [ ]:
output = tokenizer.encode("Welcome to the 🤗 Tokenizers library.")

* 絵文字が`[UNK]`トークンとしてトークン化されている。

In [ ]:
print(output.tokens)

## DataLoader

### collate関数
* サンプルに前処理を施してミニバッチを作ることを、collateする、と言う。
* collate関数の中でトークナイザを呼び出している。
* 今回は、同じミニバッチに含まれるテキストをすべてつなげてしまう。
  * `offsets`は、各テキストが、先頭から数えて何トークン目から始まるかを表す。
  * 正確には、先頭から数えて何トークン目から始まるか、マイナス１、がオフセット。
* このcollate関数は、後でDataLoaderを作るときに使う。

In [ ]:
def collate_batch(batch):
  label_list, text_list, offsets = [], [], [0]
  for instance in batch:
    _label, _text = instance["label"], instance["text"]
    # ラベルはラベルで集める
    label_list.append(_label)
    token_ids = torch.tensor(tokenizer.encode(_text).ids, dtype=torch.int64)
    # トークンidの列も集める
    text_list.append(token_ids)
    # オフセットも集める
    offsets.append(token_ids.size(0))
  label_list = torch.tensor(label_list, dtype=torch.int64)
  offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
  text_list = torch.cat(text_list)
  return label_list.to(device), text_list.to(device), offsets.to(device)

* 訓練データ、検証データ、テストデータのDataLoaderを作る。
* collate関数の使い方に注目。

### DataLoaderのインスタンスの作成

In [ ]:
from torch.utils.data import DataLoader

# ミニバッチのサイズを適当に決める
BATCH_SIZE = 64

train_dataloader = DataLoader(
    dataset["train"], batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    dataset["valid"], batch_size=BATCH_SIZE, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    dataset["test"], batch_size=BATCH_SIZE, collate_fn=collate_batch
)

In [ ]:
next(iter(train_dataloader))

## `torch.nn.EmbeddingBag`
* 全トークンのembeddingの平均を一挙に求めるlayer。

* 説明目的で、見やすいように低次元で埋め込み層を作ってみる。

In [ ]:
from torch import nn

embedding = nn.EmbeddingBag(len(vocab), 8, sparse=False)

* 本当に平均を求めているかを確認する。

In [ ]:
text = "language models"
input = torch.tensor(tokenizer.encode(text).ids, dtype=torch.int64)
offsets = torch.tensor([0], dtype=torch.int64)
embedding(input=input, offsets=offsets)

In [ ]:
tokenizer.encode(text).tokens

In [ ]:
text = "language"
input = torch.tensor(tokenizer.encode(text).ids, dtype=torch.int64)
offsets = torch.tensor([0], dtype=torch.int64)
output1 = embedding(input=input, offsets=offsets)
output1

In [ ]:
text = "models"
input = torch.tensor(tokenizer.encode(text).ids, dtype=torch.int64)
offsets = torch.tensor([0], dtype=torch.int64)
output2 = embedding(input=input, offsets=offsets)
output2

In [ ]:
(output1 + output2) / 2

* offsetsはテキストの切れ目を表す。
  * offsetsを利用すれば、複数のテキストをつなげたままベクトル化できる。
  * メモリの効率も時間的な効率も良い。

In [ ]:
text = "language models text classification"
tokenizer.encode(text).tokens

* オフセットを指定してembedする。
  * この例では、"text classification"が二つ目のテキストとなる。

In [ ]:
text = "language models text classification"
input = torch.tensor(tokenizer.encode(text).ids, dtype=torch.int64)
offsets = torch.tensor([0, 2], dtype=torch.int64)
embedding(input=input, offsets=offsets)

## 分類モデル
* `nn.Module`を継承して自前のクラスを定義する。


In [ ]:
from torch import nn

class TextClassificationModel(nn.Module):
  def __init__(self, vocab_size, embed_dim, num_class):
    super(TextClassificationModel, self).__init__()
    # 埋め込み層
    self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
    # 分類用の全結合層
    self.fc = nn.Linear(embed_dim, num_class)
    # 自前の重み初期化関数を呼び出す
    self.init_weights()

  # 自前の重み初期化関数
  def init_weights(self):
    initrange = 0.5
    self.embedding.weight.data.uniform_(-initrange, initrange)
    self.fc.weight.data.uniform_(-initrange, initrange)
    self.fc.bias.data.zero_()

  # forward pass
  def forward(self, text, offsets):
    embedded = self.embedding(text, offsets)
    return self.fc(embedded)

* 訓練データを使ってクラスの個数を調べる。

In [ ]:
unique_labels = set([label for label in dataset["train"]["label"]])
print(unique_labels)
num_class = len(unique_labels)

* 重要な定数を変数にセットする。

In [ ]:
# 語彙サイズ
vocab_size = len(vocab)

# 埋め込みベクトルの次元（これは適当に決める）
emsize = 64

* モデルのインスタンスを作成しGPUへ送る。
 * 上で値をセットした変数を使って初期化している。

In [ ]:
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

## 訓練に使うヘルパ関数

In [ ]:
import time

def train(dataloader):
  model.train()
  total_acc, total_count = 0, 0
  log_interval = 500 # ログ情報を表示する間隔
  start_time = time.time()

  for idx, (label, text, offsets) in enumerate(dataloader):
    optimizer.zero_grad()
    predicted_label = model(text, offsets)
    loss = criterion(predicted_label, label)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
    optimizer.step()
    total_acc += (predicted_label.argmax(1) == label).sum().item()
    total_count += label.size(0)
    if idx % log_interval == 0 and idx > 0:
      elapsed = time.time() - start_time
      print(
          f"||| {idx:5d}/{len(dataloader):5d} batches | "
          f"time: {elapsed:5.2f}s | "
          f"accuracy {total_acc / total_count:8.3f}"
      )
      total_acc, total_count = 0, 0
      start_time = time.time()

## 評価に使うヘルパ関数

In [ ]:
def evaluate(dataloader):
  model.eval()
  total_acc, total_count = 0, 0

  with torch.no_grad():
    for idx, (label, text, offsets) in enumerate(dataloader):
      predicted_label = model(text, offsets)
      loss = criterion(predicted_label, label)
      total_acc += (predicted_label.argmax(1) == label).sum().item()
      total_count += label.size(0)
  return total_acc / total_count

## モデルの訓練

* エポック数と学習率の設定
  * SGDを使うので、学習率は大きい目の値にしている。

In [ ]:
epochs = 10
learning_rate = 5

* 損失関数、最適化アルゴリズム、スケジューラの設定

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)


* 学習の実行

In [ ]:
total_accu = None

for epoch in range(epochs):
  epoch_start_time = time.time()
  train(train_dataloader)
  accu_val = evaluate(valid_dataloader)
  if total_accu is not None and total_accu > accu_val:
    # 検証データの正解率が前のエポックより下がったらスケジューラを動かす
    scheduler.step()
  else:
    total_accu = accu_val
  print("-" * 59)
  elapsed = time.time() - epoch_start_time
  print(
      f"| end of epoch {epoch+1:3d} | "
      f"time: {elapsed:5.2f}s | "
      f"lr = {optimizer.param_groups[0]['lr']:.3f} | "
      f"validation accuracy {accu_val:8.3f}"
  )
  print("-" * 80)

In [ ]:
torch.save(model.state_dict(), os.path.join(save_dir, "my-model.pt"))

In [ ]:
model = TextClassificationModel(vocab_size, emsize, num_class)
model.load_state_dict(torch.load(os.path.join(save_dir, "my-model.pt"), weights_only=True))
model.eval()

## 最後にテストセットで評価

In [ ]:
model.to(device)

print("Checking the results of test dataset...")
accu_test = evaluate(test_dataloader)
print(f"test accuracy {accu_test:8.3f}")

In [ ]:
def predict(text):
  with torch.no_grad():
    text = torch.tensor(tokenizer.encode(text).ids)
    output = model(text, torch.tensor([0]))
    return output.argmax(1).item()


ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

model.to("cpu")

print("This is a {} news".format(ag_news_label[predict(ex_text_str)]))

# 課題
* アーキテクチャやoptimizerやschedulerを変更して、validation set上で評価しつつモデルをチューニングしよう。
* 余裕があれば、トークナイザもチューニングしよう。
  * 例: トークン化アルゴリズムをBPEに変えてみる。
* 最後に、自分でチューニングした設定を使って、test set上で評価しよう。